In [1]:
import numpy as np
from scipy.linalg import expm
from qutip import *
from qsq_protocol import average_fidelity, plus_state, minus_state

In [2]:
# Function 1: Calculate U from theta1, theta2, theta3
def compute_U(theta1, theta2, theta3):
    # Constructing the unitary matrix U(θ1, θ2, θ3)
    U = np.array([
        [np.exp(1j * theta1) * np.cos(theta2), np.exp(1j * theta3) * np.sin(theta2)],
        [np.exp(-1j * theta3) * np.sin(theta2), np.exp(-1j * theta1) * np.cos(theta2)]
    ])
    return U

# Function 2: Given a matrix U, compute theta1, theta2, theta3
def decompose_U(U):
    # Extracting theta2 from the sine elements (sin(θ2) term)
    theta2 = np.arcsin(np.abs(U[1, 0]))  # Using the off-diagonal elements
    
    # Using the real part of U[0, 0] and U[1, 1] to extract theta1 and theta3
    cos_theta2 = np.cos(theta2)
    theta1 = np.angle(U[0, 0])  # The phase angle of U[0, 0] gives θ1
    theta3 = np.angle(U[0, 1])  # The phase angle of U[0, 1] gives θ3
    
    return theta1, theta2, theta3

# Example usage:
theta1, theta2, theta3 = np.pi/2, np.pi/3, np.pi/6
print(theta1,theta2,theta3)
U = compute_U(theta1, theta2, theta3)
print("U:", U)

# Decompose U back into theta1, theta2, theta3
theta1_reconstructed, theta2_reconstructed, theta3_reconstructed = decompose_U(U)
print(f"Reconstructed theta1: {theta1_reconstructed}, theta2: {theta2_reconstructed}, theta3: {theta3_reconstructed}")


1.5707963267948966 1.0471975511965976 0.5235987755982988
U: [[3.061617e-17+0.5j       7.500000e-01+0.4330127j]
 [7.500000e-01-0.4330127j 3.061617e-17-0.5j      ]]
Reconstructed theta1: 1.5707963267948966, theta2: 1.0471975511965976, theta3: 0.5235987755982988


In [3]:
rho_exp = rand_dm(2)
M_exp = ket2dm(rand_ket(2))
gate_exp = rand_unitary(2)
average_fidelity(rho_exp,gate_exp,M_exp)

np.float64(0.753065)

In [4]:
import numpy as np
from scipy.optimize import minimize

# Assuming you have a compute_U function and average_fidelity function defined

# Define the objective function to minimize
def objective_function(params, rho_exp, gate_exp, M_exp):
    # Unpack the parameters (theta1, theta2, theta3)
    theta1, theta2, theta3 = params
    
    # Compute the unitary operator U based on theta1, theta2, theta3
    U = Qobj(compute_U(theta1, theta2, theta3))
    
    # Compute the objective function (average fidelity)
    return 1 - average_fidelity(U @ rho_exp @ U.dag(), U @ gate_exp @  U.dag(), U @ M_exp @  U.dag())

# Example fixed matrices (rho_exp, gate_exp, M_exp)
rho_exp = rand_dm(2)
M_exp = rand_dm(2)
gate_exp = rand_unitary(2)
print(average_fidelity(rho_exp,gate_exp,M_exp))

# Initial guess for the parameters (theta1, theta2, theta3)
initial_guess = [0.1, 0.2, 0.3]

# Perform the optimization
result = minimize(
    objective_function, 
    initial_guess, 
    args=(rho_exp, gate_exp, M_exp),  # Pass the fixed matrices as extra arguments
    method='BFGS'  # You can change this to another method if needed
)

# Display the result
if result.success:
    print("Optimal parameters:", result.x)
    print("Objective function value at optimal parameters:", 1 - result.fun)
else:
    print("Optimization failed:", result.message)


0.764242
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.61498603+0.00000000e+00j 0.64368187+9.94039651e-03j]
 [0.64368187-9.94039651e-03j 0.69813129-1.38777878e-17j]]


ValueError: Invalid fidelity for M: 1.1402809004461476. Fidelity values must be between 0 and 1. Check the M fidelity calculation and the input matrix M.

In [5]:
# Define the matrix as a NumPy array
matrix = np.array([[0.61498603+0.00000000e+00j, 0.64368187+9.94039651e-03j],
                   [0.64368187-9.94039651e-03j, 0.69813129-1.38777878e-17j]])

# Convert the matrix to a Qobj
qobj_matrix = Qobj(matrix)

In [ ]:
fidelity

In [ ]:
# Initial guess for the parameters (theta1, theta2, theta3)
eigenvalues, eigenstates = M_exp.eigenstates()
U_guess = eigenstates[0] @ plus_state.dag() - 1j * eigenstates[1] @ minus_state.dag()
initial_guess = decompose_U(U_guess)

# Perform the optimization
result = minimize(
    objective_function, 
    initial_guess, 
    args=(rho_exp, gate_exp, M_exp),  # Pass the fixed matrices as extra arguments
    method='COBYLA'  # You can change this to another method if needed
)

# Display the result
if result.success:
    print("Optimal parameters:", result.x)
    print("Objective function value at optimal parameters:", 1 - result.fun)
else:
    print("Optimization failed:", result.message)


ValueError: Invalid fidelity for rho: 1.0105258499331742. Fidelity values must be between 0 and 1. Check the rho fidelity calculation and the input state.

In [ ]:
evalue[0] * estate[0] @ estate[0].dag()  + evalue[1] * estate[1] @ estate[1].dag()

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.33871197+0.00000000e+00j -0.16614519-3.76660949e-01j]
 [-0.16614519+3.76660949e-01j  0.66128803-2.77555756e-17j]]

In [ ]:
U_guess = estate[0] @ plus_state.dag() - 1j*estate[1] @ minus_state.dag()
U_guess

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[-0.58412028-0.39850157j -0.58412028+0.39850157j]
 [ 0.37360916+0.60034673j -0.69526506+0.12886618j]]